In [1]:
import os,sys
import numpy as np
import xarray as xr
import zarr
# from dask.distributed import Client
import multiprocessing.popen_spawn_posix
from distributed import Client

c = Client()
c

/gpfslocalsup/pub/anaconda-py3/2020.11/envs/climate_science/lib/python3.9/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38845 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:43703 Dashboard: http://127.0.0.1:38845/status,Cluster Workers: 5 Cores: 10 Memory: 322.12 GB


In [4]:
dir_sc = os.environ['SCRATCH']
dir_work = os.environ['WORK']

# 1) save the raw ocean bottom pressure 

In [5]:
%%time
for nexp in ["ES","EAI","EGAI"]:
    diro = dir_work+"/ETUDE3/ENS_VAR_BUDGET/raw_data/"+nexp+"/"

    ii=1
    totb=[]
    
    # load sea ice mask to apply
    SIMSK_E = xr.open_zarr(dir_work+"/ETUDE3/ENS_VAR_BUDGET/raw_data/"+nexp+"/"+nexp+"_SIMASK15_E1980_2018.zarr").SIMASK15_E #  Sea ice mask
    sum_simsk = SIMSK_E.sum(('ens','time_counter')).compute()
    si_mask = (sum_simsk.where(sum_simsk==390,0)/390)
    
    for Emb in ['001','002','003','004','005','006','007','008','009','010']:
        diridatref="/gpfsstore/rech/cli/rcli002/eORCA025.L75/"+prefix+"."+nexp+"."+Emb+"-S/1y/"
        data = xr.open_mfdataset(diridatref+'????/*_gridTsurf.nc', chunks = chunk_size, parallel=True).botpres.squeeze()
    
        totb.append(data.assign_coords({'ens':ii}))
        ii+=1
        
    OBP_E = xr.concat(totb, 'ens').sel(time_counter=slice("1980","2018")) * si_mask # full time period

    OBP_E.to_dataset(name="botpres").to_zarr(diro+"PB/raw_pb_1980_2018.zarr", mode='w')

CPU times: user 6min 29s, sys: 13.8 s, total: 6min 42s
Wall time: 11min 1s


# 2) preprocessing of the 

In [6]:
prefix = "eORCA025.L75-IMHOTEP"
chunk_size = {"x":133,"y":320}

# ---------------------------------- weight for the GMSL calculation -----------------------------------------------
diri="/gpfswork/rech/cli/rcli002/eORCA025.L75/eORCA025.L75-I/"
mesh_hgr = xr.open_dataset(diri+'mesh_hgr.nc',chunks=chunk_size).squeeze()
tmask = mesh_hgr.tmask

e1t = mesh_hgr.e1t.fillna(0)
e2t = mesh_hgr.e2t.fillna(0)

# nav_lon =  mesh_hgr.nav_lon
# nav_lat =  mesh_hgr.nav_lat

# constructing the weight
bt = (e1t * e2t).load()# area of each cell
# getting the weights to be applied to every grid cell
# masking the land cells
oceanArea = (bt*tmask[0])  #tmask : 1 on ocean, 0 on land
totalOceanArea = oceanArea.sum()
Weight = (oceanArea/totalOceanArea).compute().drop_vars('nav_lev').drop_vars('time_counter')


In [7]:
%%time

for nexp in ["ES","EAI","EGAI"]:
    diro = dir_work+"/ETUDE3/ENS_VAR_BUDGET/data4analyses/"+nexp+"/MANO/"

    OBP_E = xr.open_zarr(dir_work+"/ETUDE3/ENS_VAR_BUDGET/raw_data/"+nexp+"/PB/raw_pb_1980_2018.zarr").botpres #  Sea ice mask

    # compute the manometric sea level
    MANO_E = ( OBP_E * 10000 * si_mask ) / ( 1035 * 9.81 )
    
    # retrieving the GMmanoSL
    mean_E_GMmanoSL = (MANO_E * Weight).sum(('x','y'))
    MANO_E_corr = MANO_E - mean_E_GMmanoSL
    
    # retrieving the ensemble mean time mean at each point 
    MANO_E_corr_tmean = MANO_E_corr.mean('time_counter').mean('ens')
    MANO_E_corr_shifted = ( MANO_E_corr - MANO_E_corr_tmean ).compute()
    
    MANO_E_corr_shifted.to_dataset(name="MANO_E").to_zarr(diro+"MANO_E_2use_1980_2018.zarr", mode='w')

CPU times: user 4min 9s, sys: 39.6 s, total: 4min 48s
Wall time: 5min 23s
